<a href="https://colab.research.google.com/github/Antique-1/llm-programming1/blob/main/11_04_3_%EB%9E%AD%EC%B2%B4%EC%9D%B8_%EB%8F%84%EA%B5%AC%EB%A1%9C_%EC%97%90%EC%9D%B4%EC%A0%84%ED%8A%B8_%EB%A7%8C%EB%93%A4%EA%B8%B0_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 랭체인 도구로 에이전트 만들기
- LangChain에서 도구를 정의하고 사용하는 방법 확인
- LLM이 스스로 도구가 필요하다고 판단하고 도구 실행을 요청하는 과정 확인
- 모델 호출 -> 도구 실행 -> 모델 재호출

## 라이브러리 불러오기

In [1]:
%pip install langchain langchain_openai

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.3 MB/s eta 0:00:00


In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

In [3]:
api_key = ""

## API KEY 불러오기

In [4]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key = api_key
    )
llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='네, 잘 지냈습니다! 당신은 어떻게 지내고 계세요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 12, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CY0sZncWC1hVWqtt5JCSsteEzCWIX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7836d3af-882a-4470-a556-74c6355874a4-0', usage_metadata={'input_tokens': 12, 'output_tokens': 16, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 도구 정의
- `@tool` : 이 데코레이터를 파이썬 함수 위에 붙이면 LangChain이 이 함수를 LLM이 사용할 수 있는 도구로 반환함
- `dogString`
  - LLM이 docstring을 읽고 이 도구가 무슨 일을 하는지, 언제 이 도구를 사용해야 하는지 등 판단
  - `Args`: LLM이 timezone과 location에 어떤 값을 넣어야 할지 결정하는 데 사용

In [5]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수

    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. 타임존이 모든 지명에 대응되지 않기 때문에 이후 llm 답변 생성에 사용됨
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time


## 모델에 도구 바인딩
- llm_with_tools 객체는 이제 도구를 사용할지 말지 스스로 결정

In [6]:
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

## 1단계 : 모델의 도구 호출 결정

In [7]:
from langchain_core.messages import SystemMessage

# 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?"),
]

# llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# 생성된 llm 답변 출력
print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CGTT7yf5HGAyv6KasS7nTcym', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CY0skMH9Y9WNgWhVV22eJjO0CQWYl', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6a5ed908-6a1b-436b-a585-90586ccd1a6e-0', tool_calls

### 문제 2)
lm_with_tools 모델이 가장 처음 반환한 AIMessage에 포함된 핵심 정보는?

In [ ]:
모델이 사용해야 할 랭체인 도구의 정보가 들어있음

## 2단계 : 도구 실행
- 실제 랭체인 사용 시에는 LangChain 프레임워크가 이 1~3단계를 랭체인 에이전트로 자동화함
- `AgentExecutor`에 도구가 바인딩된 LLM과 실제 실행할 도구 목록만 전달해주면 됨

In [8]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]] # tool_dict를 사용하여 도구 함수를 선택
    print(tool_call["args"]) # 도구 호출 시 전달된 인자 출력
    tool_msg = selected_tool.invoke(tool_call) # 도구 함수를 호출하여 결과를 반환
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재시각 2025-11-04 11:16:21 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CGTT7yf5HGAyv6KasS7nTcym', 'function': {'arguments': '{"timezone":"Asia/Seoul","location":"부산"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 135, 'total_tokens': 158, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CY0skMH9Y9WNgWhVV22eJjO0CQWYl', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6a5ed908-6a1b-436b-a585-90586ccd1a6e-0', tool_cal

## 3단계 : 모델의 최종 답변

In [9]:
llm_with_tools.invoke(messages)

AIMessage(content='부산은 지금 2025년 11월 4일 오전 11시 16분입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 192, 'total_tokens': 216, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CY0sqn2Y8cJajhLenNmNrL8GugZi1', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--993e3c51-c669-4a30-81c4-c0be20fffee2-0', usage_metadata={'input_tokens': 192, 'output_tokens': 24, 'total_tokens': 216, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})